In [ ]:
!pip install -q peft transformers datasets

In [ ]:
!pip uninstall -y pyarrow
!pip install pyarrow

Found existing installation: pyarrow 18.0.0
Uninstalling pyarrow-18.0.0:
  Successfully uninstalled pyarrow-18.0.0
  Using cached pyarrow-18.0.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
Using cached pyarrow-18.0.0-cp310-cp310-manylinux_2_28_x86_64.whl (40.0 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.
pylibcudf-cu12 24.10.1 requires pyarrow<18.0.0a0,>=14.0.0, but you have pyarrow 18.0.0 which is incompatible.


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader
from tqdm import tqdm

device = 'cuda'
model_name_or_path = 'bigscience/bloomz-560m'
tokenizer_name_or_path = 'bigscience/bloomz-560m'
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    prompt_tuning_init_text='Classify if the tweet is a complaint or not:',
    tokenizer_name_or_path = model_name_or_path,
)

dataset_name = 'twitter_complaints'
checkpoint_name=f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/","_"
)
text_column="Tweet text"
label_column='text_label'
max_length=64
lr = 3e-2
num_epochs=50
batch_size=8


In [ ]:
dataset = load_dataset('ought/raft', dataset_name)
dataset['train'][0]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


{'Tweet text': '@HMRCcustomers No this is my first job', 'ID': 0, 'Label': 2}

In [ ]:
classes = [k.replace("_"," ") for k in dataset['train'].features['Label'].names]
dataset = dataset.map(
    lambda x: {'text_label': [classes[label] for label in x['Label']]},
    batched=True,
    num_proc=1,
)
dataset['train'][0]

{'Tweet text': '@HMRCcustomers No this is my first job',
 'ID': 0,
 'Label': 2,
 'text_label': 'no complaint'}

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
if tokenizer.pad_token_id is None:
  tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)['input_ids']) for class_label in classes])
print(target_max_length)

3


In [ ]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset['train'].column_names,
    load_from_cache_file=False,
    desc='Running tokenizer on dataset'
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [ ]:
train_dataset = processed_datasets['train']
eval_dataset = processed_datasets['test']

train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size,pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size,pin_memory=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0015
None


In [ ]:
optimizer = torch.optim.AdamW(model.parameters(),lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer = optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [ ]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 425/425 [01:37<00:00,  4.35it/s]


epoch=0: train_ppl=tensor(1.3732e+12, device='cuda:0') train_epoch_loss=tensor(27.9482, device='cuda:0') eval_ppl=tensor(3402.5791, device='cuda:0') eval_epoch_loss=tensor(8.1323, device='cuda:0')


100%|██████████| 425/425 [01:37<00:00,  4.35it/s]


epoch=1: train_ppl=tensor(2110.2556, device='cuda:0') train_epoch_loss=tensor(7.6546, device='cuda:0') eval_ppl=tensor(2798.1658, device='cuda:0') eval_epoch_loss=tensor(7.9367, device='cuda:0')


100%|██████████| 425/425 [01:37<00:00,  4.35it/s]


...

epoch=47: train_ppl=tensor(1.2520, device='cuda:0') train_epoch_loss=tensor(0.2247, device='cuda:0') eval_ppl=tensor(4566554.5000, device='cuda:0') eval_epoch_loss=tensor(15.3343, device='cuda:0')


100%|██████████| 425/425 [01:38<00:00,  4.33it/s]


epoch=48: train_ppl=tensor(1.2454, device='cuda:0') train_epoch_loss=tensor(0.2194, device='cuda:0') eval_ppl=tensor(4432684., device='cuda:0') eval_epoch_loss=tensor(15.3045, device='cuda:0')


100%|██████████| 425/425 [01:38<00:00,  4.33it/s]

epoch=49: train_ppl=tensor(1.2349, device='cuda:0') train_epoch_loss=tensor(0.2110, device='cuda:0') eval_ppl=tensor(4498810., device='cuda:0') eval_epoch_loss=tensor(15.3193, device='cuda:0')


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
peft_model_id = 'hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM'
model.push_to_hub('hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM' ,use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:894: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


adapter_model.safetensors:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM/commit/cec0f6cf69dab7099270f1876e02e6be6709ac00', commit_message='Upload model', commit_description='', oid='cec0f6cf69dab7099270f1876e02e6be6709ac00', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM', endpoint='https://huggingface.co', repo_type='model', repo_id='hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM'), pr_revision=None, pr_num=None)

In [ ]:
from peft import PeftModel, PeftConfig

peft_model_id = 'hang1n/bloomz-560m_PROMPT_TUNING_CAUSAL_LM'

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

adapter_config.json:   0%|          | 0.00/510 [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/32.9k [00:00<?, ?B/s]

In [ ]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"} Label : ',
    return_tensors="pt",
)

In [ ]:
model.to(device)
with torch.no_grad():
  inputs = {k: v.to(device) for k, v in inputs.items()}
  outputs = model.generate(
      input_ids = inputs['input_ids'], attention_mask = inputs['attention_mask'],max_new_tokens=10, eos_token_id=3
  )
  print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label : complaint']
